In [ ]:
import networkx as nx
import simplejson as json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# parse the gml file and build the graph object
g = nx.read_gml('networks/data/karate.gml', label='id')
# create a dictionary in a node-link format that is suitable for JSON serialization
# d = json_graph.node_link_data(g)
# with open('networks/data/power_grid_westernus.json', 'w') as fp:
#     json.dump(d, fp)

In [ ]:
nx.adjacency_data(g)

In [ ]:
nx.draw(g, with_labels=True)

In [ ]:
vote_data = pd.read_csv('networks/data/house_votes_2022_curated.csv')

In [ ]:
vote_data["DATE"] = pd.to_datetime(vote_data["DATE"])

In [ ]:
vote_data["DATE"] = vote_data["DATE"].dt.date

In [ ]:
vote_data.sort_values(by=['DATE'], inplace=True)

In [ ]:
vote_data.reset_index(drop=True, inplace=True)

In [ ]:
vote_data["REPRESENTATIVE"] = vote_data["REPRESENTATIVE"].astype(str)

In [ ]:
vote_data["index"] = np.arange(len(vote_data))

In [ ]:
yea_nay_totals = vote_data.groupby(["BILL", "VOTE"]).count()

In [ ]:
yea_nay_totals

In [ ]:
reps_voted_yea_on_same_bill = vote_data[vote_data["VOTE"] == "YEA"].groupby(["BILL", "REPRESENTATIVE", "PARTY"]).count()

In [ ]:
reps_voted_nay_on_same_bill = vote_data[vote_data["VOTE"] == "NAY"].groupby(["BILL", "REPRESENTATIVE", "PARTY"]).count()

In [ ]:
reps_voted_same_on_bill = vote_data.groupby(["BILL", "REPRESENTATIVE", "VOTE"]).count()

In [ ]:
def create_node_link_json_from_vote_data(vote_data, reps_voted_yea_on_same_bill, reps_voted_nay_on_same_bill):
    nodes = []
    links = []
    for index, row in vote_data.iterrows():
        print(index)
        nodes.append({"id": row["REPRESENTATIVE"], "group": row["PARTY"], "vote": row["VOTE"], "bill": row["BILL"], "date": row["DATE"].strftime("%Y-%m-%d")})
    for index, row in reps_voted_yea_on_same_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    for index, row in reps_voted_nay_on_same_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    return {"nodes": nodes, "links": links}

In [155]:
def nodes_links_same_votes(vote_data, reps_voted_same_on_bill):
    nodes = []
    links = []
    for index, row in vote_data.iterrows():
        nodes.append({"id": row["REPRESENTATIVE"], "group": row["PARTY"], "vote": row["VOTE"], "bill": row["BILL"], "date": row["DATE"].strftime("%Y-%m-%d")})
    for index, row in reps_voted_same_on_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    return {"nodes": nodes, "links": links}

In [156]:
net_data = nodes_links_same_votes(vote_data, reps_voted_same_on_bill)

In [158]:
net_data

{'nodes': [{'id': 'Amodei',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gibbs',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gimenez',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gohmert',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Golden',
   'group': 'Democratic',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gomez',
   'group': 'Democratic',
   'vote': 1,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gonzales, Tony',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gonzalez (OH)',
   'group': 'Republican',
   'vote': 0,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Gonzalez, Vicente',
   'group': 'Democratic',
   'vote': 1,
   'bill': 1319,
   'date': '2021-03-10'},
  {'id': 'Good (VA)',
   'group': 'Republican',

In [159]:
with open('networks/data/house_votes_2022_curated.json', 'w') as fp:
    json.dump(net_data, fp)

TypeError: Object of type int64 is not JSON serializable

In [ ]:
def convert_yea_nay_to_binary(vote_data):
    vote_data["VOTE"] = vote_data["VOTE"].apply(lambda x: 1 if x == "YEA" else 0)
    return vote_data

In [ ]:
vote_data = convert_yea_nay_to_binary(vote_data)

In [ ]:
vote_data

In [ ]:
with open('networks/data/house_votes_network.json', 'w') as fp:
    json.dump(net_data, fp)

In [ ]:
yea_nay_totals